Biblotecas necessárias

In [61]:
#Bibliotecas usadas.
from selenium import webdriver #Selenium simula uma instância do navegador.
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time #Para pausas
import os #Operações do sistema operacional.
import sys #PAssar operações fora do python no cmd do sistema.
from distutils.dir_util import copy_tree
import shutil
import pandas as pd #Organização dos dados.
import random

Tipos de prescrição desejadas (dividir em outros fatores gera arquivos do excle muito grande eo site não responde).

Definir pasta de download de dados na variável data_folder

In [15]:
# Array com os ids de utilização da anvisa, se tentamos puxar todos os dados de uma vez o site dá timeout.


data_folder='C:\Anaconda\Jupyter\Scrape\data' #Pasta para salvar os dados extraidos.
new_files = data_folder+"\\new\\"
storage = data_folder+"\\store\\"
dump_folder=data_folder+'\\new\\dump\\' #pasta do dump de dados, aqui com base na pasta dos xls, mas pode ser qualquer pasta

For loop baixando os arquivos xls de index de processo para uma pasta ./new dentro da pasta de dados definida em data_folder

In [17]:
#Loop baixando o arquivo Excel para cada id de utilização.
array=[529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,27414]
n=1 #utilizado na mensagem de progresso
for i in array:
    #url com a lista de processos
    url = 'https://consultas.anvisa.gov.br/#/medicamentos/q/?prescricoes='+str(i)
    #pasta dos arquivos .xls
    download_folder = data_folder+"\\new\\"+str(i)
    store_folder = data_folder+"\\store\\"+str(i)
    if not os.path.exists(data_folder): os.mkdir(data_folder) #cria pasta se ela não existe
    
    #opções do selenium
    d_options = webdriver.ChromeOptions()
    d_options.add_argument('--ignore-certificate-errors') #bypass certificados de segurança
    d_options.add_argument("--test-type")
    d_options.add_experimental_option("prefs", {"download.default_directory": download_folder,"download.prompt_for_download": False}) #pular confirmação de downloads
    d_options.add_argument("headless") #rodar o browser no background
    driver = webdriver.Chrome(options=d_options) #carregando o browser
    driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    #comandos possibilitando baixar arquivo em background
    params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_folder}}
    command_result = driver.execute("send_command", params)
    #broswer vai até a url
    driver.get(url)
    #mensagem de progresso
    sys.stdout.write('\r'+"Downloading initial file "+str(n)+" of "+str(len(array))+" to "+download_folder)
    sys.stdout.flush()
    time.sleep(10) #tempo para carregar a página, 10 é provavelmente exagero, uns 3 deve ser suficiente.
    #encontra o botão de download via xpath
    button = driver.find_element_by_xpath('/html/body/div[3]/div[1]/form/div/div[4]/a[1]')
    button.click() #clica no botao
    time.sleep(3) #espera o .xls baixar para continuar no próximo item do loop0
    n=n+1

Lista de processos presentes no index do arquivo xls.

In [39]:
stored = [] #variável vazia para a tabela de dados
if not os.path.exists(storage): os.mkdir(storage)
for r, d, f in os.walk(storage): #para cada .xls baixado
    for file in f:
        if ".xls" in file:
            stored0 = pd.read_excel(os.path.join(r, file)) #juntar os arquivos .xls num arquivo
            stored.append(data0) 
            
new = [] #variável vazia para a tabela de dados
for rn, dn, fn in os.walk(new_files): #para cada .xls baixado
    for file in fn:
        if ".xls" in file:
            new0 = pd.read_excel(os.path.join(rn, file)) #juntar os arquivos .xls num arquivo
            new.append(new0)
             
if stored != []:
    stored = pd.concat(stored,axis=0)
new = pd.concat(new,axis=0)

Novos processos

In [43]:
stored_p=[]
if stored != []:
    for i in stored['Processo'].values:
        stored_p.append(i.replace(".","").replace("/","").replace("-",""))
    
new_p=[]
for i in new['Processo'].values:
    value = i.replace(".","").replace("/","").replace("-","")
    if value not in stored_p:
        new_p.append(value)

In [63]:
#Obter bulas e informações para cada novo processo.
n=1    
#for i in process: #Número nem grande, quanto estiver testando utilizar um valor menor, comenta essa linha e descomenta a próxima.
sample = new_p[:3]
for i in sample: #Considera só os 3 primeiros processos.
    url2="https://consultas.anvisa.gov.br/#/medicamentos/"+str(i)+"/?numeroProcesso="+str(i) #url do processo 
    download_folder = dump_folder+str(i)
        
    #mensagem de progresso    
    sys.stdout.write('\r'+"Downloading files for process "+str(n)+" out of "+str(len(sample))+" - URL: "+url2+" - Save to: "+download_folder)
    sys.stdout.flush()
    
    #opções do browser, checar comentários do outro loop
    d_options = webdriver.ChromeOptions()
    d_options.add_argument('--ignore-certificate-errors')
    d_options.add_argument("--test-type")
    d_options.add_experimental_option("prefs", {"download.default_directory": download_folder,"download.prompt_for_download": False})
    d_options.add_argument("headless")
    driver = webdriver.Chrome(options=d_options)
    driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_folder}}
    command_result = driver.execute("send_command", params)
    driver.get(url2)
    time.sleep(3) #tempo para a pagina carregar
    
    #download do pdf das bulas
    button = driver.find_element_by_xpath('/html/body/div[3]/div[1]/form/div[1]/div[2]/table/tbody/tr[6]/td[2]/a')
    button.click()
    time.sleep(1)
    button = driver.find_element_by_xpath('/html/body/div[3]/div[1]/form/div[1]/div[2]/table/tbody/tr[6]/td[3]/a')
    button.click()
    time.sleep(1)    
    
    #botão para expandir as tabelas, não sei se é necessário.
    button = driver.find_element_by_xpath('/html/body/div[3]/div[1]/form/div[2]/a')
    button.click() #clica no botão
    time.sleep(1)
    
    #carrega código html da pagina em uma variável
    ele = driver.find_element_by_xpath('/html/body/div[3]/div[1]').get_attribute('innerHTML')
    
    #salva variável num arquivo html
    with open(download_folder+'\main.html','w') as f:
        print(ele, file=f)  
    random_time=random.randrange(5,20,1) #Tempo aleatório
    n=n+1
    #time.sleep(random_time) #Espera tempo aleatório entre 5 e 20 segundos

Move downloaded files to storage folder

In [64]:
copy_tree(new_files,storage,verbose=0)

['C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\27414\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\529\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\530\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\531\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\532\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\533\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\534\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\535\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\537\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\538\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\539\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\540\\consulta_medicamento.xls',
 'C:\\Anaconda\\Jupyter\\Scrape\\data\\store\\541\\consulta_me

Delete Download Folder

In [65]:
shutil.rmtree(new_files)